In [1]:
from gekko import GEKKO

In [2]:
import numpy as np

In [3]:
# Инициализация модели
model = GEKKO(remote=False)

In [4]:
# Решатель APOPT
model.options.SOLVER = 1

In [5]:
# Матрица трассируемости требований к ПО на файлы исходного кода
Erf = np.array([[  1,   0,   0,   0],  # Требование № 1 реализовано в файле № 1
                [0.5, 0.5,   0,   0],  # Требование № 2 реализовано в файлах № 1 и 2
                [  0, 0.5, 0.5,   0],  # Требования № 3 реализовано в файлах № 2 и 3
                [  0,   0, 0.5, 0.5],  # Требование № 4 реализовано в файлах № 3 и 4
                [  0,   0,   0,   1]]) # Требовнаие № 5 реализовано в файле № 4

In [6]:
# Матрица зависимостей между файлами исходного кода
Eff = np.array([[0, 0, 0, 0],  # На файл № 1 нет зависимостей у других файлов
                [1, 0, 0, 0],  # На файл № 2 есть зависимость у файла № 1
                [0, 0, 0, 0],  # На файл № 3 нет зависимостей у других файлов
                [0, 0, 0, 0]]) # На файл № 4 нет зависимостей у других файлов

In [7]:
# Число плагинов
k = 3

In [8]:
# Число требований(n) и число файлов исходного кода(m)
n, m = np.shape(Erf)

In [9]:
# Матрица распределения файлов по плагинам
# Необходимо определить оптимальное распределение
# Здесь указаны ограничения на максимальное и минимальное значение
# И на то, что элементы должны быть целыми числами
Efp = model.Array(model.Var, (m, k), lb=0, ub=1, integer=True)

In [10]:
# Ограничения на сумму элементов в строках
equations = []
for row in Efp:
    equation = model.sum(row) == 1
    equations.append(equation)
model.Equations(equations)

In [11]:
# Функция активации элементов на слое требований
activate_r = lambda x: 1 if x == 1 else 0

In [12]:
# Функция активации элементов на слое файлов исходного кода
activate_f = lambda x: 1 if x > 0 else 0

In [13]:
# Функция активации элементов на слое плагинов
activate_p = lambda x: 1 if x > 0 else 0

In [14]:
# Рекурсивная функция расчета зависимостей между файлами исходного кода
calculate_dependencies = lambda i, F: np.dot(Eff, F) if i == 0 else np.dot(Eff, calculate_dependencies(i - 1, F))

In [15]:
# Вспомогательная функция получения списка всех возможных комбинаций полезных требований
def parse_bin_value(bin_value):
    result = []
    parsed_value = bin_value.replace('0b', '')
    for value in parsed_value:
        int_value = int(value)
        result.append(int_value)
    return result

In [16]:
# Основная функция получения списка всех возможных комбинаций полезных требований
def collect_R_c(n):
    requirements = []
    pow_n = 2 ** n
    min_value = 1
    max_value = pow_n - 1
    bin_max_value = bin(max_value)
    parsed_max_value = parse_bin_value(bin_max_value)
    len_parsed_max_value = len(parsed_max_value)
    for value in range(min_value, pow_n):
        bin_value = bin(value)
        parsed_value = parse_bin_value(bin_value)
        len_parsed_value = len(parsed_value)
        while len_parsed_value < len_parsed_max_value:
            parsed_value.insert(0, 0)
            len_parsed_value = len(parsed_value)
        requirements.append(parsed_value)
    return requirements

In [17]:
# Функция расчета коэффициента бесполезности
def calculate_K_un(R):
    # Получение полезных файлов исходного кода
    F = np.dot(R, Erf)

    # Разрешение зависимостей
    files_count = len(F) 
    for i in range(files_count):
        dependencies = calculate_dependencies(i, F)
        F += dependencies

    # Активация элементов на слое файлов исходного кода
    F = np.array([activate_f(f) for f in F])

    # Получение плагинов, которые содержат файлы исходного кода
    P = np.dot(F, Efp)

    # Ативация элементов на слое плагинов
    for p in P:
        p.value = activate_p(p.value)

    # Получение всех файлов исходного кода, которые будут включены в поставку
    F_d = np.dot(Efp, P)

    # Активация элементов на слое файлов исходного кода
    for f_d in F_d:
        f_d.value = activate_f(f_d.value)

    # Получение всех требований, которые будут включены в поставку
    R_d = np.dot(Erf, F_d)

    # Активация элементов на слое требований
    for r_d in R_d:
        r_d.value = activate_r(r_d.value)

    # Вычисление коэффициента бесполезности
    K_un = (model.sum(R_d) - sum(R)) / (model.sum(R_d))
    
    return K_un

In [18]:
# Все возможные комбинации полезных требований (R_c - combinations of requirements)
R_c = collect_R_c(n)

In [19]:
R_c

[[0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 1, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1],
 [0, 0, 1, 1, 0],
 [0, 0, 1, 1, 1],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 1, 0],
 [0, 1, 0, 1, 1],
 [0, 1, 1, 0, 0],
 [0, 1, 1, 0, 1],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 1],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 1],
 [1, 0, 0, 1, 0],
 [1, 0, 0, 1, 1],
 [1, 0, 1, 0, 0],
 [1, 0, 1, 0, 1],
 [1, 0, 1, 1, 0],
 [1, 0, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 1],
 [1, 1, 0, 1, 0],
 [1, 1, 0, 1, 1],
 [1, 1, 1, 0, 0],
 [1, 1, 1, 0, 1],
 [1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1]]

In [20]:
# Целевая функция
obj = model.sum([calculate_K_un(R) for R in R_c]) / len(R_c)

In [21]:
# Минимизация целевой функции
model.Minimize(obj)

In [22]:
# Решение
model.solve()

apm 213.193.18.111_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            1
   Constants    :            0
   Variables    :           44
   Intermediates:            0
   Connections  :           32
   Equations    :           36
   Residuals    :           36
 
 Number of state variables:             44
 Number of total equations: -           36
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              8
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.42 NLPi:   14 Dpth:    0 Lvs:    3 Obj:  5.26E-01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.12 NLP

In [23]:
# Ответ
Efp

array([[[1.0], [0.0], [0.0]],
       [[0.0], [0.0], [1.0]],
       [[0.0], [0.0], [1.0]],
       [[0.0], [1.0], [0.0]]], dtype=object)